
# 🏠 House Prices – Full ML Pipeline

**Author:** Mateo Pascual Esseiva  

**Goal:** End-to-end, reproducible pipeline for the Kaggle challenge
**“House Prices: Advanced Regression Techniques”** — with professional structure, clear narrative, and cross-validated modeling.

**Pipeline Outline**
1) EDA & sanity checks  

2) Cleaning & context-aware imputations  

3) Outlier handling via percentile clipping  

4) Target transformation (`log1p`)  

5) Preprocessing pipeline (numeric + categorical)  

6) Cross-validated models (Ridge & LightGBM)  

7) Final training, predictions & `submission.csv`  

8) Interpretation notes & next steps


## 0. Imports & Settings

In [ ]:

import os, sys, math, json, random
import numpy as np
import pandas as pd
from pathlib import Path

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import KFold, cross_val_score, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.linear_model import Ridge

from lightgbm import LGBMRegressor

pd.options.mode.chained_assignment = None
sns.set(style="whitegrid")
RANDOM_STATE = 42


## 1. Load Data & Initial Checks


**Instructions:** place Kaggle files in `../data/` relative to this notebook:
- `train.csv`
- `test.csv`

We keep paths relative so the notebook works inside a repo.


In [ ]:

DATA_DIR = Path("../data")
train = pd.read_csv(DATA_DIR / "train.csv")
test  = pd.read_csv(DATA_DIR / "test.csv")

print("Train shape:", train.shape)
print("Test shape :", test.shape)
train.head(3)


In [ ]:

# Dtypes & missing values (top 10)
print(train.info())
(train.isnull().sum().sort_values(ascending=False).head(10) / len(train) * 100).round(2)


In [ ]:

# Duplicates
dups = train.duplicated().sum()
print(f"Duplicates in train: {dups}")


## 2. Cleaning Strategy (High-Missing Columns)


We drop columns with more than **80%** missing data as they carry little predictive signal and add noise.Known columns in this dataset: `PoolQC`, `Alley`, `Fence`, `MiscFeature`.


In [ ]:

missing_pct = train.isnull().sum() / len(train) * 100
cols_drop = missing_pct[missing_pct > 80].index.tolist()
cols_drop


In [ ]:

train.drop(columns=cols_drop, inplace=True, errors="ignore")
test.drop(columns=cols_drop, inplace=True, errors="ignore")
print("Dropped columns:", cols_drop)


## 3. Target Distribution & Quick Outlier Look

In [ ]:

fig, ax = plt.subplots(1,1, figsize=(6,4))
sns.boxplot(x=train['SalePrice'], ax=ax)
ax.set_title("SalePrice – Outlier check")
plt.show()


## 4. Context-Aware Imputations


We avoid global averages and respect domain meaning:

- **Categorical absence → `"None"`**: `Garage*`, `Bsmt*`, `FireplaceQu`, `MasVnrType`  
- **Numeric absence → `0`**: `GarageArea`, `Bsmt*SF`, `TotalBsmtSF`, `MasVnrArea`  
- **Neighborhood-aware**: `LotFrontage` → **median per `Neighborhood`**  
- **Global mode**: `Electrical`


In [ ]:

cols_none = ['GarageType','GarageFinish','GarageQual','GarageCond',
             'BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2',
             'FireplaceQu','MasVnrType']

for col in cols_none:
    if col in train.columns:
        train[col] = train[col].fillna('None')
        test[col]  = test[col].fillna('None')

cols_zero = ['GarageYrBlt','GarageArea','GarageCars','BsmtFinSF1','BsmtFinSF2',
             'BsmtUnfSF','TotalBsmtSF','MasVnrArea']

for col in cols_zero:
    if col in train.columns:
        train[col] = train[col].fillna(0)
        test[col]  = test[col].fillna(0)

# Neighborhood median for LotFrontage
if 'LotFrontage' in train.columns and 'Neighborhood' in train.columns:
    train['LotFrontage'] = train.groupby('Neighborhood')['LotFrontage'].transform(lambda x: x.fillna(x.median()))
if 'LotFrontage' in test.columns and 'Neighborhood' in test.columns:
    test['LotFrontage'] = test.groupby('Neighborhood')['LotFrontage'].transform(lambda x: x.fillna(x.median()))

# Mode for Electrical
if 'Electrical' in train.columns:
    train['Electrical'] = train['Electrical'].fillna(train['Electrical'].mode()[0])
if 'Electrical' in test.columns:
    test['Electrical'] = test['Electrical'].fillna(test['Electrical'].mode()[0])

# Sanity check
print("Remaining NaNs (train):", int(train.isnull().sum().sum()))
print("Remaining NaNs (test) :", int(test.isnull().sum().sum()))


## 5. Outliers: Percentile Clipping (P1–P99)


We prefer **winsorization** over row deletion to keep high-value properties while limiting their leverage.


In [ ]:

suspicious = ['GrLivArea', 'LotArea', 'TotalBsmtSF']
lims = {}
for c in suspicious:
    if c in train.columns:
        lo, hi = train[c].quantile(0.01), train[c].quantile(0.99)
        lims[c] = (lo, hi)

for c, (lo, hi) in lims.items():
    train[c] = train[c].clip(lower=lo, upper=hi)
    if c in test.columns:
        test[c] = test[c].clip(lower=lo, upper=hi)

lims


## 6. Target Transformation: `log1p`

In [ ]:

y_raw = train['SalePrice'].copy()
train['SalePrice_log'] = np.log1p(train['SalePrice'])

fig, ax = plt.subplots(1, 2, figsize=(12,4))
sns.histplot(y_raw, kde=True, ax=ax[0]); ax[0].set_title("SalePrice (original)")
sns.histplot(train['SalePrice_log'], kde=True, ax=ax[1]); ax[1].set_title("SalePrice_log (log1p)")
plt.tight_layout(); plt.show()


## 7. Preprocessing Pipeline (Numeric + Categorical)

In [ ]:

X = train.drop(columns=['SalePrice', 'SalePrice_log', 'Id'], errors='ignore')
y = train['SalePrice_log']

num_cols = X.select_dtypes(include=['int64','float64']).columns.tolist()
cat_cols = X.select_dtypes(include=['object']).columns.tolist()

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, num_cols),
    ('cat', categorical_transformer, cat_cols)
])
len(num_cols), len(cat_cols)


## 8. Cross-Validated Models (Ridge vs. LightGBM)

In [ ]:

models = {
    "Ridge": Ridge(alpha=1.0, random_state=RANDOM_STATE),
    "LightGBM": LGBMRegressor(random_state=RANDOM_STATE)
}

cv = KFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
scorer = make_scorer(mean_squared_error, greater_is_better=False)

def cv_rmse(model):
    pipe = Pipeline(steps=[('preprocessor', preprocessor),
                          ('model', model)])
    scores = cross_val_score(pipe, X, y, scoring=scorer, cv=cv)
    rmse = np.sqrt(-scores.mean())
    return rmse

for name, mdl in models.items():
    print(f"{name} RMSE_log: {cv_rmse(mdl):.4f}")


### (Optional) Quick Tuning for LightGBM

In [ ]:

params = {
    'model__num_leaves': [31, 64, 128],
    'model__learning_rate': [0.05, 0.1],
    'model__n_estimators': [500, 1000]
}

grid = GridSearchCV(
    Pipeline(steps=[('preprocessor', preprocessor),
                    ('model', LGBMRegressor(random_state=RANDOM_STATE))]),
    param_grid=params,
    scoring=scorer,
    cv=cv,
    n_jobs=-1
)

grid.fit(X, y)
best_rmse = np.sqrt(-grid.best_score_)
print("Best RMSE_log:", best_rmse)
print("Best params:", grid.best_params_)


## 9. Final Training & Predictions (`submission.csv`)

In [ ]:

final_model = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', LGBMRegressor(random_state=RANDOM_STATE))])

final_model.fit(X, y)

X_test = test.drop(columns=['Id'], errors='ignore')
pred_log = final_model.predict(X_test)
pred = np.expm1(pred_log)

submission = pd.DataFrame({
    'Id': test['Id'],
    'SalePrice': pred
})

out_path = Path("../submission/submission.csv")
out_path.parent.mkdir(parents=True, exist_ok=True)
submission.to_csv(out_path, index=False)
print(f"✅ Saved: {out_path.resolve()}")


## 10. Interpretation & Next Steps


**Interpretation of RMSE (log scale):**  
An RMSE_log ≈ 0.14 corresponds to ~15% relative error on price (≈ ±$27k at $180k).

**Next Steps**
- Feature importance & SHAP interpretability
- LightGBM regularization (`reg_alpha`, `reg_lambda`, `min_child_samples`)
- Early stopping with a validation split
- Potential ensembling (Ridge + LGBM + XGBoost)
